# MAC0422 - Sistemas Operacionais

### Análise de resultados do EP1

Esse notebook contém código útil para gerar arquivos de trace significativamente grandes para serem testados no simulador do EP1.

Também apresenta a implementação de análises e elaboração dos gráficos a partir dos resultados obtidos por cada um dos processos de escalonamento.

In [37]:
import numpy as np
import time
import pandas
import os

In [39]:
# pegar a pasta relativa desse notebook no seu pc

local_path = os.path.dirname(os.path.abspath("analysis.ipynb"))

print(local_path)

/home/daniel/bcc/mac422/ep1/analysis


In [26]:
# caso não tenha o pandas instalado
# import pandas
# !{sys.executable} -m pip install pandas

In [9]:
print(np.random.randint(60))

42


In [10]:
def create_trace(n_procs, max_dt, max_arrival_time, filename=''):
    if (filename == ''):
        filename = f'trace{n_procs}'
    
    tracefile = open(f'data/{filename}', 'w')
    times = []
    # generates arrival times and sorts them
    for i in range(n_procs):
        t0 = np.random.randint(max_arrival_time)
        times.append(t0)
        
    times = np.sort(times)
    
    for i in range(n_procs):
        name = f'processo{i}'
        dt = np.random.randint(1, max_dt)
        deadline = times[i] + dt + np.random.randint(n_procs - i)
        tracefile.write(f'{name} {times[i]} {dt} {deadline}\n')
    tracefile.close()


### Gerando dados

Gerar 90 traces diferentes para podermos calcular média e intervalo de confiança :p

In [16]:
for t in range(1,30):
    create_trace(n_procs=10, max_dt=5, max_arrival_time=15, filename=f'analysis_trace10_{t}')

Cria um trace com 100 processos, que demoram 5 segundos no máximo e chegam no escalonador em até 500 segundos.

In [15]:
for t in range(1,30):
    create_trace(n_procs=100, max_dt=5, max_arrival_time=50, filename=f'analysis_trace100_{t}')

Cria um trace com 1000 processos, que demoram 5 segundos no máximo e chegam no escalonador em até 5000 segundos.

In [19]:
for t in range(1,30):
    create_trace(n_procs=1000, max_dt=5, max_arrival_time=500, filename=f'analysis_trace1000_{t}')

## Resultados

O script `run_experiments.sh scheduler nprocs` executa, para o escalonador passado, 30 repetições com traces aleatórios gerados acima contendo 10, 100 ou 1000 processos (definir com nprocs).

### Arquivos gerados

Cada escalonador (s) é representado por um inteiro

1. First come first served
2. Shortest remaining time next
3. Round Robin

Cada quantidade de processos é representada pela letra p e cada uma das 30 repetições é identificada por n (n varia de 1 a 30). Assim, cada execução de `./ep1` pelo `run_experiments.sh` gera um arquivo

```
analysis/results/s_{escalonador}_p_{quantidade de processos}_n_{numero da repetição}_out
```

O conteúdo desses arquivos são p linhas contendo

```
nome_processo tf tr
```

e uma linha adicional ao final indicando a quantidade de mudanças de contexto.

## Leitura dos resultados

>observar qual foi o impacto dos diferentes escalonadores no cumprimento dos deadlines dos processos simulados e na quantidade de mudanças
de contexto.

In [46]:
scheduler = [1, 2, 3]

### Traces contendo 10 processos

arquivos de entrada

* analysis_trace10_n


arquivos de saída

* `s_1_p_10_out`
* `s_2_p_10_out`
* `s_3_p_10_out`

In [59]:
n_procs = 10
context_switches_by_scheduler = []
for s in scheduler:
    switches = []
    repetition = 1
    output_file = open(f'{local_path}/results/s_{s}_p_{n_procs}_out', 'r')
    for line in output_file:
        line_content = line.replace('\n', '').split(' ')
        # coleta mudança de contexto
        if (len(line_content) == 1):
            switches.append(int(line_content[0]))
            repetition+=1    
    context_switches_by_scheduler.append(switches)

print(context_switches_by_scheduler)

[[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [11, 11, 11, 11, 12, 11, 12, 11, 11, 12, 11, 13, 12, 12, 10, 12, 11, 13, 12, 14, 11, 11, 12, 13, 11, 13, 11, 11, 11], [27, 21, 23, 32, 18, 20, 30, 28, 22, 25, 23, 23, 26, 28, 25, 22, 24, 29, 26, 21, 27, 26, 28, 21, 33, 22, 20, 25, 21]]


#### Mudanças de contexto: